In [6]:
# Load Data 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------
# Load datasets
# ------------------------------
crime_rate_df = pd.read_csv("/workspaces/Crime-Prediction-Analysis/Dataset/Crime_Data_2023_Offenses_(With_Lat_&_Long_Info).csv")
unemp_df_2023 = pd.read_csv("/workspaces/Crime-Prediction-Analysis/Dataset/SYRA036URN_2023.csv")
crime_rate_df_2 = pd.read_csv("/workspaces/Crime-Prediction-Analysis/Dataset/Crime_Data_2024_Offenses_With_Lat_and_Long_Info.csv")  # Load the new crime dataset
unemp_df_2024 = pd.read_csv("/workspaces/Crime-Prediction-Analysis/Dataset/SYRA036URN_2024.csv")

# ------------------------------
# Parse and align date formats
# ------------------------------
crime_rate_df['DATEEND'] = pd.to_datetime(crime_rate_df['DATEEND'], errors='coerce')
crime_rate_df['month'] = crime_rate_df['DATEEND'].dt.to_period('M').dt.to_timestamp()

unemp_df_2023['observation_date'] = pd.to_datetime(unemp_df_2023['observation_date'])
unemp_df_2023.rename(columns={'observation_date': 'month', 'SYRA036URN': 'unemployment_rate'}, inplace=True)

unemp_df_2024['observation_date'] = pd.to_datetime(unemp_df_2024['observation_date'])
unemp_df_2024.rename(columns={'observation_date': 'month', 'SYRA036URN': 'unemployment_rate'}, inplace=True)

# Parse and align the date for the new crime dataset
crime_rate_df_2['DATEEND'] = pd.to_datetime(crime_rate_df_2['DATEEND'], errors='coerce')
crime_rate_df_2['month'] = crime_rate_df_2['DATEEND'].dt.to_period('M').dt.to_timestamp()

# ------------------------------
# Merge datasets
# ------------------------------
# Merge the first crime dataset and unemployment data
crime_df_1 = pd.merge(crime_rate_df, unemp_df_2023, on='month', how='left')

# Merge the second crime dataset
crime_df_2 = pd.merge(crime_rate_df_2, unemp_df_2024, on='month', how='left')

# Combine the two crime datasets
crime_df = pd.concat([crime_df_1, crime_df_2], ignore_index=True)

# Export the final merged dataset to a CSV file
crime_df.to_csv("/workspaces/Crime-Prediction-Analysis/Dataset/newly_2324.csv", index=False)

# ------------------------------
# Initial cleaning - Removing rows with missing values
# ------------------------------
crime_df_new = crime_df.drop_duplicates()


# Drop rows where LarcenyCode is missing
crime_df_new = crime_df_new.dropna(subset=['LarcenyCode'])


# Remove rows with missing values in 'LAT', 'LONG', and 'unemployment_rate'
crime_df_new = crime_df_new.dropna(subset=['LAT', 'LONG'])

# Ensure numeric types, removing rows with invalid LAT/LONG
crime_df_new['LAT'] = pd.to_numeric(crime_df_new['LAT'], errors='coerce')
crime_df_new['LONG'] = pd.to_numeric(crime_df_new['LONG'], errors='coerce')

# Remove rows where LAT or LONG are still NaN after coercion
crime_df_new = crime_df_new.dropna(subset=['LAT', 'LONG'])

# ------------------------------
# Clean 'Arrest' column
# ------------------------------
if 'Arrest' in crime_df.columns:
    crime_df_new['Arrest'] = crime_df_new['Arrest'].fillna('NO').astype(str).str.strip().str.upper()

# ------------------------------
# Encode crime type
# ------------------------------
crime_type_col = None
for col in ['CODE DEFINED', 'CODE_DEFINED', 'Crm Cd Desc', 'Crm Cd']:
    if col in crime_df_new.columns:
        crime_type_col = col
        break

if crime_type_col:
    crime_df_new['CrimeType'] = crime_df_new[crime_type_col].astype('category').cat.codes
else:
    crime_df_new['CrimeType'] = -1  # fallback if not found

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------
# Exploratory Data Analysis (EDA)
# ------------------------------

# Assuming crime_df and unemp_df are already loaded

# Step 1: Convert 'DATEEND' to datetime if it's not already
crime_df_new['DATEEND'] = pd.to_datetime(crime_df_new['DATEEND'])

# Step 3: Data types and missing values
print("\nData types and missing values:")
print(crime_df_new.info())


# Step 6: Remove outliers based on IQR method for unemployment_rate and crime_count
def remove_outliers(df, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Calculate the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the data to remove outliers
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers for both 'unemployment_rate' and 'crime_count'
crime_df_new = remove_outliers(crime_df_new, 'unemployment_rate')

# Step 7: Recalculate monthly crime counts after removing outliers
crime_df_new['month'] = crime_df_new['DATEEND'].dt.to_period('M')
monthly_crime_counts = crime_df_new.groupby('month').size().reset_index(name='crime_count')

# Step 8: Merge the monthly crime counts back to the original DataFrame using 'month' as the key
# Rename 'crime_count' in the monthly_crime_counts to avoid conflicts during the merge
monthly_crime_counts.rename(columns={'crime_count': 'monthly_crime_count'}, inplace=True)

# Merge crime_df with monthly_crime_counts, ensuring that 'crime_count' doesn't conflict
crime_df_new = pd.merge(crime_df_new, monthly_crime_counts, on='month', how='left')
crime_df_new.to_csv("/workspaces/Crime-Prediction-Analysis/Dataset/df_new_exp.csv", index=False) 

# Step 9: Verify the DataFrame now contains the 'crime_count' column
print(crime_df_new[['DATEEND', 'month', 'monthly_crime_count']].head())  # Check the first few rows to ensure the column is added


crime_df_new.to_csv("/workspaces/Crime-Prediction-Analysis/Dataset/crime_df_2324_un.csv", index=False)



/tmp/ipykernel_10844/2558836453.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  crime_rate_df['month'] = crime_rate_df['DATEEND'].dt.to_period('M').dt.to_timestamp()
/tmp/ipykernel_10844/2558836453.py:28: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  crime_rate_df_2['month'] = crime_rate_df_2['DATEEND'].dt.to_period('M').dt.to_timestamp()



Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
Index: 10945 entries, 0 to 13760
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   X                  10945 non-null  float64            
 1   Y                  10945 non-null  float64            
 2   DATEEND            10945 non-null  datetime64[ns, UTC]
 3   TIMESTART          10945 non-null  object             
 4   TIMEEND            10945 non-null  int64              
 5   ADDRESS            10945 non-null  object             
 6   Arrest             10945 non-null  object             
 7   LarcenyCode        10945 non-null  object             
 8   CODE_DEFINED       10945 non-null  object             
 9   LAT                10945 non-null  float64            
 10  LONG               10945 non-null  float64            
 11  ObjectId           10945 non-null  int64              
 12  month              

/tmp/ipykernel_10844/2558836453.py:121: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  crime_df_new['month'] = crime_df_new['DATEEND'].dt.to_period('M')


                    DATEEND    month  monthly_crime_count
0 2023-04-27 00:00:00+00:00  2023-04                  331
1 2023-09-16 00:00:00+00:00  2023-09                  574
2 2023-12-01 00:00:00+00:00  2023-12                  381
3 2023-06-10 00:00:00+00:00  2023-06                  349
4 2023-08-24 00:00:00+00:00  2023-08                  608
